# We will try with 

https://pytorch.org/docs/stable/generated/torch.trapz.html

https://pytorch.org/docs/stable/generated/torch.trapezoid.html#torch.trapezoid

in the Integral Riccati:

$$ \psi = I_\alpha (\lambda \psi^2 + \mu \psi + \nu) = I_\alpha(g)$$

(if the initial condition $I_{1-\alpha} \psi(0) = 0$.) where 

$$g = \lambda \psi^2 + \mu \psi + \nu$$

We will compute $I_\alpha(\lambda \psi^2 + \mu \psi + \nu)$ throught the trapezioid. We will use $\psi_t$ as 
a function of the neural network. 

We can use $\psi_t := x^\alpha N(x)$ 

In [1]:
import torch
import torch.nn as nn
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
T = 1/252
alph = 0.64
lamb = 0.045
mu = -64.938
nu = 0

In [3]:
N = nn.Sequential(nn.Linear(1, 30), nn.Sigmoid(),nn.Linear(30,1, bias=False))


Psi_t = lambda x: x**alph * N(x)  # Trial solution

### We remind the fractional R-L integral is defined as:

$$I_\alpha g(t) = \frac{1}{\Gamma(\alpha)}\int_0^t (t-s)^{\alpha -1} g(s) ds $$

# Test With Trapezoid. Doesn't work

In [4]:
n_points = 50
x_train = np.linspace(0, T, n_points)[:, None]
x = torch.Tensor(x_train)
x.requires_grad = True


n_rectangles = 50
x_in_the_integral = np.linspace(0, T, n_rectangles)[:, None]
x_in_the_integral = torch.Tensor(x_in_the_integral)
x_in_the_integral.requires_grad = True

def I_alp(t):
    ''' Be careful, t cannot be a tensor with more than 1 dimension. So we will use this fact in loss function'''
    x_truncated_up_to_t = x_in_the_integral[x_in_the_integral<=t]
    shape = x_truncated_up_to_t.shape[0]
    x_trun = torch.reshape(x_truncated_up_to_t, (shape,1))
    
    # print("x_trun =", x_trun)
    
    y = Psi_t(x_trun)
    # print("y=", y)
    
    g = lamb*y**2 + mu*y + nu
    # print("g =",g)
    integrand = (t-x_trun)**(alph-1)*g
    integral = torch.trapezoid(integrand[:,0], x_trun[:,0])
    
    result = 1/math.gamma(alph)*integral
    return result

In [5]:
I_alp(0.0001)

tensor(2.7571e-05, grad_fn=<MulBackward0>)

In [6]:
# x_in_the_integral[x_in_the_integral<= x[3]]        OK
# x_in_the_integral[x_in_the_integral<= x]           ERROR

In [7]:
def loss(x):
    
    outputs = Psi_t(x).T
    integrals_vector = torch.tensor([[I_alp(i) for i in x]])

    final_loss = torch.mean((outputs - integrals_vector)**2)
    
    print('loss is', final_loss)

    return final_loss

In [8]:
optimizer = torch.optim.Adadelta(N.parameters())

def closure():

    optimizer.zero_grad()
    l = loss(x)
    l.backward()
    
    return l

In [9]:
# Here train the neural network
for i in range(10):
    optimizer.step(closure)

loss is tensor(inf, grad_fn=<MeanBackward0>)
loss is tensor(nan, grad_fn=<MeanBackward0>)
loss is tensor(nan, grad_fn=<MeanBackward0>)
loss is tensor(nan, grad_fn=<MeanBackward0>)
loss is tensor(nan, grad_fn=<MeanBackward0>)
loss is tensor(nan, grad_fn=<MeanBackward0>)
loss is tensor(nan, grad_fn=<MeanBackward0>)
loss is tensor(nan, grad_fn=<MeanBackward0>)
loss is tensor(nan, grad_fn=<MeanBackward0>)
loss is tensor(nan, grad_fn=<MeanBackward0>)


# Comparing results with Euler

In [10]:
euler_df = pd.read_csv('results_from_tests/for_test_conformable_nn.csv').set_index('time')
euler_df.head()

,value
time,
0.000000,0.000000
0.000124,0.007018
0.000248,0.010921
0.000372,0.014140
0.000496,0.016982


In [ ]:
with torch.no_grad():
    yy = Psi_t(x).detach().numpy()
y_euler = euler_df.value
x_euler = euler_df.index

fig, ax = plt.subplots(dpi=100)
ax.plot(x_euler, y_euler, label='True')
ax.plot(x_train, yy, 'x', label='Neural network approximation')
ax.set_xlabel('$x$')
ax.set_ylabel('$Psi(x)$')
plt.legend(loc='best');